In [4]:
!pip install fuzzywuzzy python-Levenshtein

  Using cached fuzzywuzzy-0.18.0-py2.py3-none-any.whl (18 kB)
  Using cached python_levenshtein-0.27.1-py3-none-any.whl (9.4 kB)
  Using cached levenshtein-0.27.1-cp311-cp311-win_amd64.whl (100 kB)
  Using cached rapidfuzz-3.13.0-cp311-cp311-win_amd64.whl (1.6 MB)



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import joblib
import pandas as pd
import os
import re # Import thư viện regex


# --- 1. Tải LabelEncoder đã lọc và Tạo Danh sách Tên Bệnh Chuẩn ---

# Điều chỉnh đường dẫn này cho phù hợp với cấu trúc thư mục của bạn
model_dir = '../models/' # Nếu notebook của bạn ở trong thư mục 'notebooks/'
label_encoder_path = os.path.join(model_dir, 'disease_label_encoder_filtered.pkl')

# Biến để lưu trữ tên bệnh chuẩn
CANONICAL_DISEASE_NAMES = []
CANONICAL_DISEASE_NAMES_SET = set()

try:
    label_encoder_filtered = joblib.load(label_encoder_path)
    print(f"Đã tải thành công LabelEncoder từ: {label_encoder_path}")

    # Lấy danh sách tên bệnh gốc từ label encoder
    raw_canonical_disease_names = label_encoder_filtered.classes_
    
    # Hàm chuẩn hóa tên (như đã định nghĩa trước)
    def standardize_name(name):
        if pd.isna(name): # Xử lý trường hợp giá trị là NaN
            return None
        s = str(name).strip().lower()
        s = re.sub(r'\s+', '_', s)
        s = re.sub(r'[^a-z0-9_]', '', s)
        s = re.sub(r'_+', '_', s)
        s = s.strip('_')
        if not s: # Nếu sau khi làm sạch, chuỗi rỗng (ví dụ: tên chỉ toàn ký tự đặc biệt)
            return None
        return s

    # Áp dụng chuẩn hóa và tạo danh sách tên bệnh chuẩn
    processed_names = []
    for name in raw_canonical_disease_names:
        standardized = standardize_name(name)
        if standardized: # Chỉ thêm nếu tên sau chuẩn hóa không rỗng
            processed_names.append(standardized)
            
    CANONICAL_DISEASE_NAMES = sorted(list(set(processed_names))) # Lấy duy nhất và sắp xếp
    CANONICAL_DISEASE_NAMES_SET = set(CANONICAL_DISEASE_NAMES)
    
    print(f"\nSố lượng tên bệnh chuẩn (từ LabelEncoder, sau khi chuẩn hóa): {len(CANONICAL_DISEASE_NAMES)}")
    if CANONICAL_DISEASE_NAMES:
        print("Một vài ví dụ tên bệnh chuẩn (đã được chuẩn hóa thêm):")
        for i, name in enumerate(CANONICAL_DISEASE_NAMES[:15]): # In ra 15 tên đầu
            print(f"  {i+1}. {name}")
    else:
        print("Không có tên bệnh chuẩn nào được tạo. Kiểm tra lại LabelEncoder.")

except FileNotFoundError:
    print(f"LỖI: Không tìm thấy file LabelEncoder tại {label_encoder_path}.")
    print("Hãy đảm bảo bạn đã chạy notebook tiền xử lý dữ liệu cho model ML và đã lưu file 'disease_label_encoder_filtered.pkl' vào đúng đường dẫn.")
except Exception as e:
    print(f"Có lỗi xảy ra khi tải hoặc xử lý LabelEncoder: {e}")

# Bây giờ bạn đã có CANONICAL_DISEASE_NAMES và CANONICAL_DISEASE_NAMES_SET
# để sử dụng cho các bước tiếp theo.

Đã tải thành công LabelEncoder từ: ../models/disease_label_encoder_filtered.pkl

Số lượng tên bệnh chuẩn (từ LabelEncoder, sau khi chuẩn hóa): 754
Một vài ví dụ tên bệnh chuẩn (đã được chuẩn hóa thêm):
  1. abdominal_aortic_aneurysm
  2. abdominal_hernia
  3. abscess_of_nose
  4. abscess_of_the_lung
  5. abscess_of_the_pharynx
  6. acanthosis_nigricans
  7. acariasis
  8. achalasia
  9. acne
  10. actinic_keratosis
  11. acute_bronchiolitis
  12. acute_bronchitis
  13. acute_bronchospasm
  14. acute_fatty_liver_of_pregnancy_aflp
  15. acute_glaucoma


In [6]:
# --- 2. Xử lý Dataset Phụ trợ: DiseaseAndSymptoms.csv ---

# Đặt đúng đường dẫn đến file của bạn
# Giả sử bạn đã tải file này về và đặt trong một thư mục, ví dụ 'data/other_source_datasets/'
other_dataset_path_1 = '../data/raw/DiseaseAndSymptoms.csv' # THAY THẾ BẰNG ĐƯỜNG DẪN THỰC TẾ

print(f"\n--- Bắt đầu xử lý dataset: {other_dataset_path_1} ---")

try:
    df_other_1 = pd.read_csv(other_dataset_path_1)
    print(f"Đã tải thành công: {other_dataset_path_1}")

    # Xác định cột chứa tên bệnh (GIẢ SỬ là cột 'Disease')
    disease_column_name_other_1 = 'Disease' # THAY THẾ NẾU TÊN CỘT KHÁC

    if disease_column_name_other_1 in df_other_1.columns:
        raw_other_1_disease_names = df_other_1[disease_column_name_other_1].dropna().unique()
        
        standardized_other_1_disease_names = sorted(list(set(
            standardize_name(name) for name in raw_other_1_disease_names if standardize_name(name)
        )))
        
        print(f"Số lượng tên bệnh duy nhất (đã chuẩn hóa) trong '{os.path.basename(other_dataset_path_1)}': {len(standardized_other_1_disease_names)}")
        if standardized_other_1_disease_names:
            print("Một vài ví dụ tên bệnh đã chuẩn hóa từ dataset này:")
            for i, name in enumerate(standardized_other_1_disease_names[:10]):
                print(f"  {i+1}. {name}")

        # --- Bắt đầu quá trình ánh xạ cho dataset này ---
        mapping_dict_other_1 = {} 
        needs_manual_review_other_1 = []
        
        print("\nBắt đầu so khớp và tạo mapping...")
        
        # Cài đặt thư viện fuzzywuzzy nếu chưa có: pip install fuzzywuzzy python-Levenshtein
        try:
            from fuzzywuzzy import process
            FUZZYWUZZY_AVAILABLE = True
        except ImportError:
            print("Thư viện fuzzywuzzy chưa được cài đặt. Sẽ bỏ qua fuzzy matching. Để cài: pip install fuzzywuzzy python-Levenshtein")
            FUZZYWUZZY_AVAILABLE = False

        for other_name in standardized_other_1_disease_names:
            if other_name in CANONICAL_DISEASE_NAMES_SET:
                mapping_dict_other_1[other_name] = other_name # Khớp trực tiếp
            elif FUZZYWUZZY_AVAILABLE:
                # Thử fuzzy matching nếu không khớp trực tiếp
                match, score = process.extractOne(other_name, CANONICAL_DISEASE_NAMES) # Tìm trong list tên chuẩn
                if score >= 90: # Ngưỡng tương đồng cao (bạn có thể điều chỉnh)
                    print(f"  Đề xuất mapping (Fuzzy >90%): '{other_name}'  =>  '{match}' (Score: {score}%)")
                    mapping_dict_other_1[other_name] = match
                else:
                    # print(f"  Không khớp rõ ràng cho '{other_name}' (Fuzzy best: '{match}' với score {score}%)")
                    needs_manual_review_other_1.append(other_name)
            else:
                needs_manual_review_other_1.append(other_name)
        
        print(f"\nĐã tạo mapping cho {len(mapping_dict_other_1)} tên bệnh.")
        if needs_manual_review_other_1:
            print(f"Cần xem xét ánh xạ thủ công cho {len(needs_manual_review_other_1)} tên bệnh sau từ '{os.path.basename(other_dataset_path_1)}':")
            for name in needs_manual_review_other_1:
                print(f"  - {name}")
        else:
            print(f"Tuyệt vời! Tất cả các tên bệnh từ '{os.path.basename(other_dataset_path_1)}' đã được ánh xạ (hoặc không cần fuzzy matching nếu không có fuzzywuzzy).")

        # (Tùy chọn) Lưu mapping_dict_other_1 này lại
        mapping_file_path_other_1 = f'../data/processed/mapping_{os.path.basename(other_dataset_path_1)}.json'
        import json
        with open(mapping_file_path_other_1, 'w', encoding='utf-8') as f:
             json.dump(mapping_dict_other_1, f, indent=2, ensure_ascii=False)
        print(f"Đã lưu mapping cho {os.path.basename(other_dataset_path_1)} vào {mapping_file_path_other_1}")

    else:
        print(f"LỖI: Không tìm thấy cột '{disease_column_name_other_1}' trong {other_dataset_path_1}")

except FileNotFoundError:
    print(f"LỖI: Không tìm thấy file {other_dataset_path_1}")
except Exception as e:
    print(f"Có lỗi xảy ra khi xử lý {other_dataset_path_1}: {e}")


--- Bắt đầu xử lý dataset: ../data/raw/DiseaseAndSymptoms.csv ---
Đã tải thành công: ../data/raw/DiseaseAndSymptoms.csv
Số lượng tên bệnh duy nhất (đã chuẩn hóa) trong 'DiseaseAndSymptoms.csv': 41
Một vài ví dụ tên bệnh đã chuẩn hóa từ dataset này:
  1. acne
  2. aids
  3. alcoholic_hepatitis
  4. allergy
  5. arthritis
  6. bronchial_asthma
  7. cervical_spondylosis
  8. chicken_pox
  9. chronic_cholestasis
  10. common_cold

Bắt đầu so khớp và tạo mapping...
  Đề xuất mapping (Fuzzy >90%): 'arthritis'  =>  'arthritis_of_the_hip' (Score: 90%)
  Đề xuất mapping (Fuzzy >90%): 'bronchial_asthma'  =>  'asthma' (Score: 90%)
  Đề xuất mapping (Fuzzy >90%): 'cervical_spondylosis'  =>  'spondylosis' (Score: 90%)
  Đề xuất mapping (Fuzzy >90%): 'chicken_pox'  =>  'chickenpox' (Score: 95%)
  Đề xuất mapping (Fuzzy >90%): 'dengue'  =>  'dengue_fever' (Score: 90%)
  Đề xuất mapping (Fuzzy >90%): 'diabetes'  =>  'diabetes_insipidus' (Score: 90%)
  Đề xuất mapping (Fuzzy >90%): 'fungal_infection' 

In [7]:
# (Tiếp tục trong notebook nơi bạn đã có df_other_1, mapping_dict_other_1, 
#  và hàm standardize_name, CANONICAL_DISEASE_NAMES_SET)

# --- Áp dụng mapping để chuẩn hóa tên bệnh trong df_other_1 ---
if 'df_other_1' in locals() and 'mapping_dict_other_1' in locals():
    # Hàm để áp dụng mapping, sử dụng lại hàm standardize_name cho tên gốc trước khi tra cứu
    def get_canonical_name(raw_disease_name, mapping_dict, canonical_set):
        standardized_raw_name = standardize_name(raw_disease_name)
        if standardized_raw_name:
            return mapping_dict.get(standardized_raw_name) # Trả về tên chuẩn nếu có trong mapping, ngược lại là None
        return None

    # Tạo cột mới 'Canonical_Disease'
    disease_column_name_other_1 = 'Disease' # Tên cột bệnh gốc trong df_other_1
    df_other_1['Canonical_Disease'] = df_other_1[disease_column_name_other_1].apply(
        lambda x: get_canonical_name(x, mapping_dict_other_1, CANONICAL_DISEASE_NAMES_SET)
    )

    # Xem xét các bệnh không map được (nếu có)
    unmapped_diseases_in_df = df_other_1[df_other_1['Canonical_Disease'].isnull()][disease_column_name_other_1].unique()
    if len(unmapped_diseases_in_df) > 0:
        print(f"\nCẢNH BÁO: {len(unmapped_diseases_in_df)} tên bệnh trong '{os.path.basename(other_dataset_path_1)}' không thể ánh xạ sang tên chuẩn và sẽ bị bỏ qua khi tạo luật:")
        for name in unmapped_diseases_in_df:
            print(f"  - {name}")
    
    # Lọc bỏ các hàng không có tên bệnh chuẩn
    df_other_1_mapped = df_other_1.dropna(subset=['Canonical_Disease']).copy()
    print(f"\nSố lượng hàng sau khi giữ lại các bệnh đã được ánh xạ: {len(df_other_1_mapped)}")
    print("Ví dụ dữ liệu với tên bệnh đã chuẩn hóa:")
    print(df_other_1_mapped[['Disease', 'Canonical_Disease', 'Symptom_1', 'Symptom_2']].head())

else:
    print("DataFrame df_other_1 hoặc mapping_dict_other_1 chưa được tạo.")


CẢNH BÁO: 13 tên bệnh trong 'DiseaseAndSymptoms.csv' không thể ánh xạ sang tên chuẩn và sẽ bị bỏ qua khi tạo luật:
  - GERD
  - Chronic cholestasis
  - Peptic ulcer diseae
  - AIDS
  - Paralysis (brain hemorrhage)
  - Typhoid
  - hepatitis A
  - Hepatitis B
  - Hepatitis C
  - Hepatitis E
  - Alcoholic hepatitis
  - Dimorphic hemmorhoids(piles)
  - (vertigo) Paroymsal  Positional Vertigo

Số lượng hàng sau khi giữ lại các bệnh đã được ánh xạ: 3360
Ví dụ dữ liệu với tên bệnh đã chuẩn hóa:
            Disease             Canonical_Disease   Symptom_1  \
0  Fungal infection  fungal_infection_of_the_hair     itching   
1  Fungal infection  fungal_infection_of_the_hair   skin_rash   
2  Fungal infection  fungal_infection_of_the_hair     itching   
3  Fungal infection  fungal_infection_of_the_hair     itching   
4  Fungal infection  fungal_infection_of_the_hair     itching   

               Symptom_2  
0              skin_rash  
1   nodal_skin_eruptions  
2   nodal_skin_eruptions  
3      

In [9]:
# --- Thu thập và chuẩn hóa triệu chứng cho từng bệnh chuẩn ---
# disease_symptom_map_from_other_1 sẽ lưu: {canonical_disease_name: set_of_standardized_symptoms}
disease_symptom_map_from_other_1 = {}

if 'df_other_1_mapped' in locals():
    for canonical_disease, group in df_other_1_mapped.groupby('Canonical_Disease'):
        all_symptoms_for_this_disease = set()
        symptom_cols = [col for col in df_other_1_mapped.columns if col.startswith('Symptom_')]
        
        for idx, row in group.iterrows(): # Lặp qua từng hàng của cùng một bệnh (nếu có nhiều)
            for col in symptom_cols:
                raw_symptom = row[col]
                if pd.notna(raw_symptom):
                    standardized_symptom = standardize_name(raw_symptom) # Dùng lại hàm standardize_name
                    if standardized_symptom: # Chỉ thêm nếu không rỗng sau chuẩn hóa
                        all_symptoms_for_this_disease.add(standardized_symptom)
        
        if all_symptoms_for_this_disease: # Chỉ thêm nếu có triệu chứng
             disease_symptom_map_from_other_1[canonical_disease] = sorted(list(all_symptoms_for_this_disease))

    print(f"\nĐã tạo xong disease_symptom_map từ '{os.path.basename(other_dataset_path_1)}'.")
    print(f"Số lượng bệnh có thông tin triệu chứng: {len(disease_symptom_map_from_other_1)}")

    # In ra ví dụ
    count = 0
    for disease, symptoms in disease_symptom_map_from_other_1.items():
        print(f"\nBệnh Chuẩn: {disease}")
        print(f"  Triệu chứng (đã chuẩn hóa): {symptoms[:5]}") # In 5 triệu chứng đầu
        count += 1
        if count >= 5:
             break
else:
    print("DataFrame df_other_1_mapped chưa được tạo.")


Đã tạo xong disease_symptom_map từ 'DiseaseAndSymptoms.csv'.
Số lượng bệnh có thông tin triệu chứng: 27

Bệnh Chuẩn: acne
  Triệu chứng (đã chuẩn hóa): ['blackheads', 'pus_filled_pimples', 'scurring', 'skin_rash']

Bệnh Chuẩn: allergy
  Triệu chứng (đã chuẩn hóa): ['chills', 'continuous_sneezing', 'shivering', 'watering_from_eyes']

Bệnh Chuẩn: arthritis_of_the_hip
  Triệu chứng (đã chuẩn hóa): ['movement_stiffness', 'muscle_weakness', 'painful_walking', 'stiff_neck', 'swelling_joints']

Bệnh Chuẩn: asthma
  Triệu chứng (đã chuẩn hóa): ['breathlessness', 'cough', 'family_history', 'fatigue', 'high_fever']

Bệnh Chuẩn: chickenpox
  Triệu chứng (đã chuẩn hóa): ['fatigue', 'headache', 'high_fever', 'itching', 'lethargy']


In [17]:
# --- Định nghĩa luật linh động hơn ---
generated_rules_flexible = [] # Đổi tên biến để phân biệt
rule_id_counter = 1

if 'disease_symptom_map_from_other_1' in locals():
    for disease_name_std, symptoms_std_list in disease_symptom_map_from_other_1.items():
        if not symptoms_std_list or len(symptoms_std_list) < 2: # Bỏ qua nếu có ít hơn 2 triệu chứng (khó đặt N trong M)
            print(f"Bỏ qua bệnh '{disease_name_std}' do có quá ít triệu chứng ({len(symptoms_std_list)}) để tạo luật linh động.")
            continue

        # Chiến lược chọn M (tất cả các triệu chứng có sẵn từ dataset này cho bệnh đó)
        all_available_symptoms = symptoms_std_list 
        m_count = len(all_available_symptoms)

        # Chiến lược chọn N (ví dụ: khoảng 50-70% của M, nhưng không ít hơn 2)
        # Bạn có thể làm phức tạp hơn, ví dụ dựa trên tầm quan trọng của triệu chứng
        if m_count >= 5:
            n_min_match = 3 # Yêu cầu ít nhất 3 nếu có 5 triệu chứng trở lên
        elif m_count >= 3:
            n_min_match = 2 # Yêu cầu ít nhất 2 nếu có 3-4 triệu chứng
        else: # m_count = 2
            n_min_match = 2 # Yêu cầu cả 2 nếu chỉ có 2 triệu chứng

        # Tạo luật với cấu trúc mới
        rule = {
            "id": f"RULE_FLEX_DS_{rule_id_counter:03d}",
            "source_dataset": os.path.basename(other_dataset_path_1),
            "if_symptoms_list": all_available_symptoms, # Danh sách M triệu chứng
            "min_symptoms_match": n_min_match,         # Số N triệu chứng cần khớp
            # "if_symptoms_must_have": {"some_key_symptom": 1}, # Tùy chọn: thêm các triệu chứng bắt buộc
            "then_disease": disease_name_std,
            "confidence": 0.65,  # Cần một cách xác định confidence tốt hơn
                                # Có thể dựa trên tỷ lệ N/M hoặc tần suất thực tế
            "explanation": f"Having at least {n_min_match} of the following symptoms: {', '.join(all_available_symptoms)} can be indicative of '{disease_name_std}'."
        }
        generated_rules_flexible.append(rule)
        rule_id_counter += 1

    print(f"\nĐã tạo ví dụ {len(generated_rules_flexible)} luật LINH ĐỘNG từ '{os.path.basename(other_dataset_path_1)}'.")
    if generated_rules_flexible:
         print("Ví dụ luật linh động đầu tiên:")
         print(json.dumps(generated_rules_flexible[0], indent=2, ensure_ascii=False))

    # --- LƯU CÁC LUẬT LINH ĐỘNG NÀY VÀO FILE JSON ---
    # Đặt tên file khác để không ghi đè file luật cũ (nếu có)
    rules_output_path_flexible = f"../data/knowledge_base/flexible_rules_from_{os.path.basename(other_dataset_path_1)}.json"
    try:
        # Đảm bảo thư mục tồn tại
        os.makedirs(os.path.dirname(rules_output_path_flexible), exist_ok=True)
        with open(rules_output_path_flexible, 'w', encoding='utf-8') as f:
             json.dump(generated_rules_flexible, f, indent=2, ensure_ascii=False)
        print(f"Đã lưu các luật linh động vào: {rules_output_path_flexible}")
    except Exception as e:
        print(f"Lỗi khi lưu file luật: {e}")
else:
    print("disease_symptom_map_from_other_1 chưa được tạo.")


Đã tạo ví dụ 27 luật LINH ĐỘNG từ 'DiseaseAndSymptoms.csv'.
Ví dụ luật linh động đầu tiên:
{
  "id": "RULE_FLEX_DS_001",
  "source_dataset": "DiseaseAndSymptoms.csv",
  "if_symptoms_list": [
    "blackheads",
    "pus_filled_pimples",
    "scurring",
    "skin_rash"
  ],
  "min_symptoms_match": 2,
  "then_disease": "acne",
  "confidence": 0.65,
  "explanation": "Having at least 2 of the following symptoms: blackheads, pus_filled_pimples, scurring, skin_rash can be indicative of 'acne'."
}
Đã lưu các luật linh động vào: ../data/knowledge_base/flexible_rules_from_DiseaseAndSymptoms.csv.json
